# Notebook to fit amplitudes to Lensing Signals

This was the program used in the paper

Runs the ampltidues for wtot, Wsys, and no weight.

Makes the files called "amplitudes"

e.g.  :

LOWZ_0.15_0.31_cweight_wtot.txt => these are the weights being printed out

LOWZ_0.15_0.31_amplitude_errs_wtot.txt => errors on amplitudes

LOWZ_0.15_0.31_amplitudes_wtot.txt => these are the amplitudes

In [1]:
import numpy as np
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [2]:
# The HOD model file
# see interpolate.pro
modelfile = "hod_model_saito2016_mdpl2.txt"       # model for weighting
temp = np.genfromtxt(modelfile)
model = temp[:,1]
rbins = temp[:,0]

In [3]:
# *** Change here depending on weight type ***
# Run once for each option
#weight_type = "_wtot"
weight_type = "_now"

In [4]:
smallr ,= np.where(rbins<1.) # 1 Mpc split
larger ,= np.where(rbins>1.)
allr ,= np.where(rbins<50)

binranges = []
binranges.append(smallr)
binranges.append(larger)
binranges.append(allr)

In [5]:
# Function that computes amplitudes

def amplitudes(measurements, covariances, model, binranges, experiments, file1, file2, file3, file4):
    # measurements is a dictionary of Nd vectors, 
    # covariances is a dictionary of NxN matrixes, 
    # model is a Nd vector, 
    # binranges is a list of integer vectors, 
    # experiments is a list of names 
    
    for binrange in binranges:
        
        print("------ Bin range -----",binrange)
        print("bin range ",binrange)
        
        # pick relevant entries from measurements, covariances, model
        meas = {}
        cov = {}
        invcov = {}
        weight = {} # optimal weight for that survey
        denom  = {} # the denominator for that survey that brings A=1 if meas=mod
        mod = model[binrange]
        #print("model",mod)
        
        # sum of covariances of that set of bins
        sumcov = np.zeros((len(binrange),len(binrange)))
        
        #print("\n----- Calculating weights -------\n")
            
        for experiment in experiments:
            
            #print(experiment)
            meas[experiment] = measurements[experiment][binrange]
            cov[experiment]  = covariances[experiment][binrange][:,binrange]
            invcov[experiment] = np.linalg.inv(cov[experiment])
            weight[experiment] = np.dot(invcov[experiment],mod)
            print("experiment",experiment)
            #type(weight[experiment])
            print("weight",weight[experiment])
            
            denom[experiment] = np.dot(weight[experiment],mod)
            
            # sum all covs for joint weighting
            sumcov += cov[experiment]
            
        # optimal weight for comparison is inverse of sum of covariances times model
        cweight = np.dot(np.linalg.inv(sumcov),mod)
        cdenom   = np.dot(cweight,mod) # normalizing the amplitude to 1 if measurement = model
        
        print("cweight",cweight)
        print("cdenom",cdenom)      
        
        #print("\n----- Calculating amplitudes -------\n")
        
        for experiment in experiments:
            a    = np.dot(meas[experiment],cweight)/cdenom
            siga = np.sqrt(np.dot(np.dot(cweight,cov[experiment]),cweight)/cdenom**2)
            #print(experiment,a,"+-",siga)
            
            # Write out amplitudes
            file1.write('%0.4f' % a)
            file1.write(' ')

            # Write out errors
            file2.write('%0.4f' % siga)
            file2.write(' ')
        
            #print("a and siga", a, siga)
   
            #print("with its own optimal weighting the experiment",experiment,"would have given us",a,"+-",siga)
            a    = np.dot(meas[experiment],weight[experiment])/denom[experiment]
            siga = np.sqrt(np.dot(np.dot(weight[experiment],cov[experiment]),weight[experiment])/denom[experiment]**2)
   
        # also print out the weights
        # to check if they have imporatnt variations with R
        #print('cweight',cweight)
        
        for item in cweight:
            file3.write("%0.4f" % item)
            file3.write(' ')
            
        file4.write('%0.4f' % cdenom)
                
        file1.write('\n')
        file2.write('\n')
        file3.write('\n')
        file4.write('\n')
            
    file1.close()  
    file2.close() 
    file3.close()
    file4.close()

In [6]:
# This one reuses an exsiting weight function from a previous calculation

def amplitudes_existingwht(measurements, covariances, model, binranges, experiments, file1, file2,cwhtfile,denomfile):
    # measurements is a dictionary of Nd vectors, 
    # covariances is a dictionary of NxN matrixes, 
    # model is a Nd vector, 
    # binranges is a list of integer vectors, 
    # experiments is a list of names 
    
    # Reuse cweight and cdenom from fiducial runs
    # Read in the files here
    
    # This file has lines of different lengths
    # Read line by line instead
    fp = open(cwhtfile)    
    cdenomarray = np.genfromtxt(denomfile)
        
    i=0
    
    for binrange in binranges:
        
        # pick relevant entries from measurements, covariances, model
        meas = {}
        cov = {}
        invcov = {}
        weight = {} # optimal weight for that survey
        denom  = {} # the denominator for that survey that brings A=1 if meas=mod
        mod = model[binrange]

        # cweight depends on radial range
        # Read in line by line here

        line = fp.readline()
        x=line.split()
        n=len(x)

        cweight=np.zeros(n)
        for j in range(0,n):
            cweight[j]=float(x[j])
        
        print("cweights",cweight)
        cdenom = cdenomarray[i]
        i=i+1
        
        for experiment in experiments:
            
            meas[experiment] = measurements[experiment][binrange]
            cov[experiment]  = covariances[experiment][binrange][:,binrange]
            invcov[experiment] = np.linalg.inv(cov[experiment])
        
        for experiment in experiments:
        
            a    = np.dot(meas[experiment],cweight)/cdenom
            siga = np.sqrt(np.dot(np.dot(cweight,cov[experiment]),cweight)/cdenom**2)
            #print(experiment,a,"+-",siga)
            
            # Write out amplitudes
            file1.write('%0.4f' % a)
            file1.write(' ')

            # Write out errors
            file2.write('%0.4f' % siga)
            file2.write(' ')
                
        file1.write('\n')
        file2.write('\n')
         
    file1.close()  
    file2.close() 

# Order of Experiments

In [7]:
# define filenames / filename parts for various runs
experiments = ["DES","CFHT","HSC","CS82","KiDS","SDSS"] 

# LOWZ (includes SDSS)

In [8]:
sample = ["LOWZ",0.15,0.31]

measurements = {}
covariances = {}
           
print('\n ---- Files names ---- \n')
    
for experiment in experiments:
    fcov = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+"_cov.dat"
    print(fcov)
    covariances[experiment] = np.genfromtxt(fcov)

    # Make sure read in DS here, there are many columns in the DS results files
    fdat = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+".dat"
    print(fdat)
    temp = np.genfromtxt(fdat)
    
    measurements[experiment]= temp[:,1] # second column in the data file
    
print('\n ---- Now run the amplitudes ---- \n')

# Now run the Amplitudes
# Where the amplitudes are recorded
file1 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes"+str(weight_type)+".txt",'w')
file1.write('# Amplitudes \n')
file1.write('# cols: DES, CFHT, HSC, CS82, KiDS, SDSS \n')
file1.write('# row 1: small R, row2: large R, row3: all R \n')

# Amplitude errors
file2 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes_errs"+str(weight_type)+".txt",'w')
file2.write('# Errors on amplitudes \n')
file2.write('# cols: DES, CFHT, HSC, CS82, KiDS, SDSS \n')
file2.write('# row 1: small R, row2: large R, row3: all R \n')

# Weights
file3 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cweight"+str(weight_type)+".txt",'w')
file4 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cdenom"+str(weight_type)+".txt",'w')

amplitudes(measurements,covariances,model,binranges,experiments,file1,file2,file3,file4)  
print("done !")


 ---- Files names ---- 

DES_LOWZ_0.15_0.31_now_cov.dat
DES_LOWZ_0.15_0.31_now.dat
CFHT_LOWZ_0.15_0.31_now_cov.dat
CFHT_LOWZ_0.15_0.31_now.dat
HSC_LOWZ_0.15_0.31_now_cov.dat
HSC_LOWZ_0.15_0.31_now.dat
CS82_LOWZ_0.15_0.31_now_cov.dat
CS82_LOWZ_0.15_0.31_now.dat
KiDS_LOWZ_0.15_0.31_now_cov.dat
KiDS_LOWZ_0.15_0.31_now.dat
SDSS_LOWZ_0.15_0.31_now_cov.dat
SDSS_LOWZ_0.15_0.31_now.dat

 ---- Now run the amplitudes ---- 

------ Bin range ----- [0 1 2 3 4]
bin range  [0 1 2 3 4]
experiment DES
weight [0.9427903  0.39640953 3.64434849 3.39777425 9.16475361]
experiment CFHT
weight [ 0.8910712   1.48028017  3.49457106  5.89931824 12.40119522]
experiment HSC
weight [0.8257893  3.4025805  5.61558152 9.24173614 6.56430587]
experiment CS82
weight [0.88758571 0.66293366 6.7918467  8.74897424 1.61974881]
experiment KiDS
weight [0.33697312 0.98030992 0.75214993 3.24391725 6.11679265]
experiment SDSS
weight [ 2.2644035   3.10432532  4.42198205 11.96382759 19.23987811]
cweight [0.12412083 0.22202148 0.47

In [9]:
sample = ["LOWZ",0.31,0.43]

measurements = {}
covariances = {}
           
print('\n ---- Files names ---- \n')
    
for experiment in experiments:
    fcov = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+"_cov.dat"
    print(fcov)
    covariances[experiment] = np.genfromtxt(fcov)

    # Make sure read in DS here, there are many columns in the DS results files
    fdat = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+".dat"
    print(fdat)
    temp = np.genfromtxt(fdat)
    
    measurements[experiment]= temp[:,1] # second column in the data file
    
print('\n ---- Now run the amplitudes ---- \n')

# Now run the Amplitudes
# Where the amplitudes are recorded
file1 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes"+str(weight_type)+".txt",'w')
file1.write('# Amplitudes \n')
file1.write('# cols: DES, CFHT, HSC, CS82, KiDS, SDSS \n')
file1.write('# row 1: small R, row2: large R, row3: all R \n')

# Amplitude errors
file2 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes_errs"+str(weight_type)+".txt",'w')
file2.write('# Errors on amplitudes \n')
file2.write('# cols: DES, CFHT, HSC, CS82, KiDS, SDSS \n')
file2.write('# row 1: small R, row2: large R, row3: all R \n')

# Weights
file3 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cweight"+str(weight_type)+".txt",'w')
file4 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cdenom"+str(weight_type)+".txt",'w')

amplitudes(measurements,covariances,model,binranges,experiments,file1,file2,file3,file4)  
print("done !")


 ---- Files names ---- 

DES_LOWZ_0.31_0.43_now_cov.dat
DES_LOWZ_0.31_0.43_now.dat
CFHT_LOWZ_0.31_0.43_now_cov.dat
CFHT_LOWZ_0.31_0.43_now.dat
HSC_LOWZ_0.31_0.43_now_cov.dat
HSC_LOWZ_0.31_0.43_now.dat
CS82_LOWZ_0.31_0.43_now_cov.dat
CS82_LOWZ_0.31_0.43_now.dat
KiDS_LOWZ_0.31_0.43_now_cov.dat
KiDS_LOWZ_0.31_0.43_now.dat
SDSS_LOWZ_0.31_0.43_now_cov.dat
SDSS_LOWZ_0.31_0.43_now.dat

 ---- Now run the amplitudes ---- 

------ Bin range ----- [0 1 2 3 4]
bin range  [0 1 2 3 4]
experiment DES
weight [0.23098219 0.84407394 0.6974741  1.72237283 3.14267052]
experiment CFHT
weight [0.5978637  1.16371371 1.86143239 3.73483442 7.4463898 ]
experiment HSC
weight [0.63597403 1.20858094 4.45462748 3.92055819 7.70879439]
experiment CS82
weight [0.43025415 0.65124786 0.91869889 1.99876799 3.30456473]
experiment KiDS
weight [0.18590698 0.40518356 0.79218838 1.92108353 3.94749952]
experiment SDSS
weight [ 0.48154437  0.65451498  3.43006957  4.11446477 14.57703012]
cweight [0.06030773 0.1215074  0.2060096

# CMASS - no SDSS here

In [10]:
experiments = ["DES","CFHT","HSC","CS82","KiDS"] 

In [11]:
sample = ["CMASS",0.43,0.54]

measurements = {}
covariances = {}
           
print('\n ---- Files names ---- \n')
    
for experiment in experiments:
    fcov = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+"_cov.dat"
    print(fcov)
    covariances[experiment] = np.genfromtxt(fcov)

    # Make sure read in DS here, there are many columns in the DS results files
    fdat = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+".dat"
    print(fdat)
    temp = np.genfromtxt(fdat)
    
    measurements[experiment]= temp[:,1] # second column in the data file
    
print('\n ---- Now run the amplitudes ---- \n')

# Now run the Amplitudes
# Where the amplitudes are recorded
file1 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes"+str(weight_type)+".txt",'w')
file1.write('# Amplitudes \n')
file1.write('# cols: DES, CFHT, HSC, CS82, KiDS \n')
file1.write('# row 1: small R, row2: large R, row3: all R \n')

# Amplitude errors
file2 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes_errs"+str(weight_type)+".txt",'w')
file2.write('# Errors on amplitudes \n')
file2.write('# cols: DES, CFHT, HSC, CS82, KiDS \n')
file2.write('# row 1: small R, row2: large R, row3: all R \n')

# Weights
file3 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cweight"+str(weight_type)+".txt",'w')
file4 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cdenom"+str(weight_type)+".txt",'w')

amplitudes(measurements,covariances,model,binranges,experiments,file1,file2,file3,file4)  
print("done !")



 ---- Files names ---- 

DES_CMASS_0.43_0.54_now_cov.dat
DES_CMASS_0.43_0.54_now.dat
CFHT_CMASS_0.43_0.54_now_cov.dat
CFHT_CMASS_0.43_0.54_now.dat
HSC_CMASS_0.43_0.54_now_cov.dat
HSC_CMASS_0.43_0.54_now.dat
CS82_CMASS_0.43_0.54_now_cov.dat
CS82_CMASS_0.43_0.54_now.dat
KiDS_CMASS_0.43_0.54_now_cov.dat
KiDS_CMASS_0.43_0.54_now.dat

 ---- Now run the amplitudes ---- 

------ Bin range ----- [0 1 2 3 4]
bin range  [0 1 2 3 4]
experiment DES
weight [0.14618139 0.49570665 0.60474932 2.23957144 4.13765715]
experiment CFHT
weight [0.44466912 1.1722521  2.26696187 4.87079642 8.93361692]
experiment HSC
weight [ 1.16420337  2.34248376  2.32691643  6.33999078 10.04489678]
experiment CS82
weight [0.23610675 0.11816258 0.864616   1.96460524 3.1255154 ]
experiment KiDS
weight [0.31085109 0.66117091 0.75294528 1.56024108 3.52405987]
cweight [0.06000993 0.10224777 0.19588494 0.50524257 0.91357899]
cdenom 63.70722208531147
------ Bin range ----- [5 6 7 8 9]
bin range  [5 6 7 8 9]
experiment DES
weight 

In [12]:
sample = ["CMASS",0.54,0.70]

measurements = {}
covariances = {}
           
print('\n ---- Files names ---- \n')
    
for experiment in experiments:
    fcov = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+"_cov.dat"
    print(fcov)
    covariances[experiment] = np.genfromtxt(fcov)

    # Make sure read in DS here, there are many columns in the DS results files
    fdat = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+".dat"
    print(fdat)
    temp = np.genfromtxt(fdat)
    
    measurements[experiment]= temp[:,1] # second column in the data file
    
print('\n ---- Now run the amplitudes ---- \n')

# Now run the Amplitudes
# Where the amplitudes are recorded
file1 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes"+str(weight_type)+".txt",'w')
file1.write('# Amplitudes \n')
file1.write('# cols: DES, CFHT, HSC, CS82, KiDS \n')
file1.write('# row 1: small R, row2: large R, row3: all R \n')

# Amplitude errors
file2 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes_errs"+str(weight_type)+".txt",'w')
file2.write('# Errors on amplitudes \n')
file2.write('# cols: DES, CFHT, HSC, CS82, KiDS \n')
file2.write('# row 1: small R, row2: large R, row3: all R \n')

# Weights
file3 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cweight"+str(weight_type)+".txt",'w')
file4 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cdenom"+str(weight_type)+".txt",'w')

amplitudes(measurements,covariances,model,binranges,experiments,file1,file2,file3,file4)  
print("done !")


 ---- Files names ---- 

DES_CMASS_0.54_0.7_now_cov.dat
DES_CMASS_0.54_0.7_now.dat
CFHT_CMASS_0.54_0.7_now_cov.dat
CFHT_CMASS_0.54_0.7_now.dat
HSC_CMASS_0.54_0.7_now_cov.dat
HSC_CMASS_0.54_0.7_now.dat
CS82_CMASS_0.54_0.7_now_cov.dat
CS82_CMASS_0.54_0.7_now.dat
KiDS_CMASS_0.54_0.7_now_cov.dat
KiDS_CMASS_0.54_0.7_now.dat

 ---- Now run the amplitudes ---- 

------ Bin range ----- [0 1 2 3 4]
bin range  [0 1 2 3 4]
experiment DES
weight [0.16598613 0.14110274 0.5833853  0.9403292  1.27805637]
experiment CFHT
weight [0.37719184 0.72706922 1.7165884  3.39967939 6.69568068]
experiment HSC
weight [ 1.26248238  4.32667883  6.41650973 18.92774501 25.19298341]
experiment CS82
weight [0.13157966 0.02367088 0.42745329 0.9333257  1.8913697 ]
experiment KiDS
weight [0.1720472  0.3065244  0.55474349 0.92335605 1.77183429]
cweight [0.04162655 0.03926306 0.14281172 0.2805751  0.48892558]
cdenom 36.456693827154226
------ Bin range ----- [5 6 7 8 9]
bin range  [5 6 7 8 9]
experiment DES
weight [ 2.77335

# Post blinding tests

In [13]:
# Nstar internal tests from Song using HSC
#HSC_CMASS_0.54_0.70_wtot_nstar_low.dat
#HSC_CMASS_0.43_0.54_wtot_nstar_low_cov.dat	
#HSC_CMASS_0.54_0.70_wtot_nstar_low_cov.dat
#HSC_CMASS_0.43_0.54_wtot_nstar_upp.dat		
#HSC_CMASS_0.54_0.70_wtot_nstar_upp.dat
#HSC_CMASS_0.43_0.54_wtot_nstar_upp_cov.dat	
#HSC_CMASS_0.54_0.70_wtot_nstar_upp_cov.dat

experiments = ["HSClow","HSCupp","HSCall"] 
sample = ["CMASS",0.54,0.70]

measurements = {}
covariances = {}
           
print('\n ---- Files names ---- \n')
    
for i in range(0,3):
  
    if(i==0):
        fcov = "HSC_CMASS_0.54_0.70_wtot_nstar_low_cov.dat"
        fdat = "HSC_CMASS_0.54_0.70_wtot_nstar_low.dat"
    
    if(i==1):
        fcov = "HSC_CMASS_0.54_0.70_wtot_nstar_upp_cov.dat"
        fdat = "HSC_CMASS_0.54_0.70_wtot_nstar_upp.dat"
        
    if(i==2):
        fcov = "HSC_CMASS_0.54_0.7_wtot_cov.dat"
        fdat = "HSC_CMASS_0.54_0.7_wtot.dat"
        
    print("fcov",fcov)
    covariances[experiments[i]] = np.genfromtxt(fcov)

    print(fdat)
    temp = np.genfromtxt(fdat)
    
    measurements[experiments[i]]= temp[:,1] # second column in the data file
    
print('\n ---- Now run the amplitudes ---- \n')

# Now run the Amplitudes
# Where the amplitudes are recorded
file1 = open("HSC_CMASS_0.54_0.70_nstartests_amplitudes.txt",'w')
file1.write('# Amplitudes \n')
file1.write('# cols: low nstar, high nstar, all \n')
file1.write('# row 1: small R, row2: large R, row3: all R \n')

# Amplitude errors
file2 = open("HSC_CMASS_0.54_0.70_nstartests_amplitudes_errs.txt",'w')
file2.write('# Errors on amplitudes \n')
file2.write('# cols: low nstar, high nstar \n')
file2.write('# row 1: small R, row2: large R, row3: all R \n')

# Resuse existing weight here!!
file3 = sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cweight"+str(weight_type)+".txt"
file4 = sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cdenom"+str(weight_type)+".txt"

# This uses existing weights computed previoulsy
amplitudes_existingwht(measurements,covariances,model,binranges,experiments,file1,file2,file3,file4)  
print("done !")


 ---- Files names ---- 

fcov HSC_CMASS_0.54_0.70_wtot_nstar_low_cov.dat
HSC_CMASS_0.54_0.70_wtot_nstar_low.dat
fcov HSC_CMASS_0.54_0.70_wtot_nstar_upp_cov.dat
HSC_CMASS_0.54_0.70_wtot_nstar_upp.dat
fcov HSC_CMASS_0.54_0.7_wtot_cov.dat
HSC_CMASS_0.54_0.7_wtot.dat

 ---- Now run the amplitudes ---- 

cweights [0.0416 0.0393 0.1428 0.2806 0.4889]
cweights [0.8341 1.0499 1.4163 1.3279 3.2526]
cweights [0.0353 0.0312 0.148  0.2651 0.463  0.8427 0.4931 1.3405 1.1655 3.1194]
done !


In [14]:
# post-blinding test
# Adding in SDSS for C1 

In [15]:
# SDSS_CMASS_0.43_0.54_wtot.dat
# SDSS_CMASS_0.43_0.54_wtot_cov.dat

experiments = ["SDSS"] 
sample = ["CMASS",0.54,0.70]

measurements = {}
covariances = {}
           
print('\n ---- Files names ---- \n')
    
i=0

fcov = "SDSS_CMASS_0.43_0.54_wtot_cov.dat"
fdat = "SDSS_CMASS_0.43_0.54_wtot.dat"
    
print("fcov",fcov)
covariances[experiments[i]] = np.genfromtxt(fcov)

print(fdat)
temp = np.genfromtxt(fdat)
    
measurements[experiments[i]]= temp[:,1] # second column in the data file
    
print('\n ---- Now run the amplitudes ---- \n')

# Now run the Amplitudes
# Where the amplitudes are recorded
file1 = open("SDSS_CMASS_0.43_0.54_amplitudes.txt",'w')
file1.write('# Amplitudes \n')
file1.write('# cols: low z, high z, all \n')
file1.write('# row 1: small R, row2: large R, row3: all R \n')

# Amplitude errors
file2 = open("SDSS_CMASS_0.43_0.54_amplitudes_errs.txt",'w')
file2.write('# Errors on amplitudes \n')
file2.write('# cols: low z, high z \n')
file2.write('# row 1: small R, row2: large R, row3: all R \n')

# Resuse existing weight here!!
file3 = sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cweight"+str(weight_type)+".txt"
file4 = sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cdenom"+str(weight_type)+".txt"

# This uses existing weights computed previoulsy
amplitudes_existingwht(measurements,covariances,model,binranges,experiments,file1,file2,file3,file4)  
print("done !")


 ---- Files names ---- 

fcov SDSS_CMASS_0.43_0.54_wtot_cov.dat
SDSS_CMASS_0.43_0.54_wtot.dat

 ---- Now run the amplitudes ---- 

cweights [0.0416 0.0393 0.1428 0.2806 0.4889]
cweights [0.8341 1.0499 1.4163 1.3279 3.2526]
cweights [0.0353 0.0312 0.148  0.2651 0.463  0.8427 0.4931 1.3405 1.1655 3.1194]
done !
